In [ ]:
import os
import json
import tarfile
import torch
from tqdm import tqdm
from glob import glob

In [ ]:
# Function to save a dictionary to a JSON file
def load_from_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

In [ ]:
# Load JSON files from a folder and parse them into a list of dictionaries
def load_json_files(folder_path, output_folder):
    # create an empty list to store the JSON data
    json_data = []
    # put the list on the device
    json_data = json_data.to(device)
    # list all files in the folder
    for filename in os.listdir(folder_path):
        # check if the file is a JSON file
        if filename.endswith(".json"):
            # build the full file path 
            file_path = os.path.join(folder_path, filename)
            # open the file and load the JSON data
            with open(file_path, 'r') as f:
                data = json.load(f)
                data = data.to(device)
                json_data.append(data)
                print("     " + filename)

    # save the loaded JSON data to a new file in the output folder
    file = os.path.join(output_folder, "output.json")
    with open(file, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

In [ ]:
# extract the tar files from the directory
def extract_tar(tar_path, extract_path):
    # Ensure the extract path exists
    os.makedirs(extract_path, exist_ok=True)

    # Open and extract the tar file
    with tarfile.open(tar_path, 'r') as tar:
        tar.extractall(path=extract_path)
    # return the path of the tar file
    return tar_path


In [ ]:
# Function to convert time strings to frames based on a given FPS
def time_to_frames(start_time_str, end_time_str, fps):

    # Split the time string into hours, minutes, and seconds
    hours, minutes, seconds = start_time_str.split(':')
    # Convert to float
    hours = float(hours)
    minutes = float(minutes)
    seconds = float(seconds)

    # Convert to total seconds
    start_frame = hours * 3600 + minutes * 60 + seconds * fps

    # Split the time string into hours, minutes, and seconds
    hours, minutes, seconds = end_time_str.split(':')
    # Convert to float
    hours = float(hours)
    minutes = float(minutes)
    seconds = float(seconds)

    # Convert to total seconds
    end_frame = hours * 3600 + minutes * 60 + seconds
    
    # Return the start and end frames
    return start_frame, end_frame

In [ ]:
def compile_json_from_tar(tar_gz_path):
    # Open the tar.gz file
    with tarfile.open(tar_gz_path, 'r:gz') as tar:
        # Extract the folder name to use as the name for the output JSON file
        folder_name = os.path.splitext(os.path.basename(tar_gz_path))[0][:-4]
        output_json_filename = f"<path_to_directory>\\{folder_name}.json"

        # List to store combined JSON data
        combined_json_data = []

        # Iterate over each file in the tar.gz archive
        for member in tar.getmembers():
            # Check if the member is a file and ends with .json
            if member.isfile() and member.name.endswith('.json'):
                # Extract the file and read the JSON data
                with tar.extractfile(member) as f:
                    try:
                        print(member.name)
                        # Load the JSON file and append it to the combined list
                        json_data = json.load(f)
                        combined_json_data.append(json_data)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON from {member.name}: {e}")

        # Write all the combined JSON data into a single output file
        with open(output_json_filename, 'w') as json_file:
            json.dump(combined_json_data, json_file, indent=4)

        print(f"Combined JSON data saved to {output_json_filename}")


In [ ]:
# Function to extract keypoints from the JSON data
def extract_keypoints(json_data):
    # Initialize a list to hold all keypoints for all frames
    keypoints_list = []
    # Iterate through each frame in the JSON data
    for frame in json_data:
        # List to hold keypoints for each frame
        frame_keypoints = [] 
        # Iterate through each person in the frame
        for person in frame["people"]:
            # To hold all keypoints for this person
            keypoints_person = []  

            # Process body keypoints
            body_keypoints = []
            # Iterate through the pose keypoints (x, y, confidence)
            for i in range(0, len(person["pose_keypoints_2d"]), 3):
                # Check if the index is within bounds
                if i+1 < len(person["pose_keypoints_2d"]) and i+2 < len(person["pose_keypoints_2d"]): 
                    # Extract x, y, and confidence values
                    x = person["pose_keypoints_2d"][i]
                    y = person["pose_keypoints_2d"][i+1]
                    confidence = person["pose_keypoints_2d"][i+2]
                    # Append the keypoint to the list
                    body_keypoints.append([x, y, confidence]) 
                else:
                    # Handle missing values by padding with zeros
                    body_keypoints.append([0, 0, 0])  
            # Append the body keypoints to the person's keypoints
            keypoints_person.append(body_keypoints)
            
            # Process left hand keypoints 
            left_hand_keypoints = []
            # Check if left hand keypoints are available
            if "hand_left_keypoints_2d" in person:
                # Iterate through the left hand keypoints (x, y, confidence)
                for i in range(0, len(person["hand_left_keypoints_2d"]), 3): 
                    # Check if the index is within bounds
                    if i+1 < len(person["hand_left_keypoints_2d"]) and i+2 < len(person["hand_left_keypoints_2d"]):
                        # Extract x, y, and confidence values
                        x = person["hand_left_keypoints_2d"][i]
                        y = person["hand_left_keypoints_2d"][i+1]
                        confidence = person["hand_left_keypoints_2d"][i+2]
                        # Append the keypoint to the list
                        left_hand_keypoints.append([x, y, confidence])
                    else:
                        # Handle missing values by padding with zeros
                        left_hand_keypoints.append([0, 0, 0]) 
                # Append the left hand keypoints to the person's keypoints
                keypoints_person.append(left_hand_keypoints)
            
            # Process right hand keypoints (if available)
            right_hand_keypoints = []
            # Check if right hand keypoints are available
            if "hand_right_keypoints_2d" in person:
                # Iterate through the right hand keypoints (x, y, confidence)
                for i in range(0, len(person["hand_right_keypoints_2d"]), 3):
                    # Check if the index is within bounds
                    if i+1 < len(person["hand_right_keypoints_2d"]) and i+2 < len(person["hand_right_keypoints_2d"]):
                        # Extract x, y, and confidence values
                        x = person["hand_right_keypoints_2d"][i]
                        y = person["hand_right_keypoints_2d"][i+1]
                        confidence = person["hand_right_keypoints_2d"][i+2]
                        # Append the keypoint to the list
                        right_hand_keypoints.append([x, y, confidence])
                    else:
                        # Handle missing values by padding with zeros
                        right_hand_keypoints.append([0, 0, 0])
                # Append the right hand keypoints to the person's keypoints
                keypoints_person.append(right_hand_keypoints)

            # Add the keypoints for this person to the frame's keypoints
            frame_keypoints.append(keypoints_person)
        
        # Add all the frame's keypoints to the main list
        keypoints_list.append(frame_keypoints)
    # Return the list of keypoints for all frames
    return keypoints_list

In [ ]:
# Function to normalize keypoints data
def normalize_keypoints(keypoints_data):
    # Initialize a list to hold all keypoints
    flattened_keypoints = []

    # Flatten all keypoints across frames with a progress bar
    for frame_keypoints in tqdm(keypoints_data, desc="Normalizing keypoints (flattening)", total=len(keypoints_data), ncols=100):
        # Iterate through each person in the frame
        for person_keypoints in frame_keypoints:
            # Iterate through each set of body keypoints for the person
            for body_keypoints in person_keypoints:
                # Iterate through each keypoint (x, y, confidence)
                for keypoint in body_keypoints:
                    # Pad keypoints to size 3 if they are not of size 3
                    if len(keypoint) < 3:
                        keypoint = keypoint + [0] * (3 - len(keypoint))  
                    # Append the keypoint to the flattened list
                    flattened_keypoints.append(keypoint)

    # Convert flattened keypoints list into a tensor for normalization
    flattened_keypoints = torch.tensor(flattened_keypoints).float()

    # Calculate mean and std for normalization
    mean = flattened_keypoints.mean(dim=0)
    std = flattened_keypoints.std(dim=0)

    # Normalize the keypoints with a progress bar
    normalized_keypoints = []
    # Iterate through each frame's keypoints
    for frame_keypoints in tqdm(keypoints_data, desc="Applying normalization", total=len(keypoints_data), ncols=100):
        # List to hold normalized keypoints for this frame
        normalized_frame_keypoints = []
        # Iterate through each person in the frame
        for person_keypoints in frame_keypoints:
            # List to hold normalized keypoints for this person
            normalized_person_keypoints = []
            # Iterate through each set of body keypoints for the person
            for body_keypoints in person_keypoints:
                # List to hold normalized keypoints for this body
                normalized_body_keypoints = []
                # Iterate through each keypoint (x, y, confidence)
                for keypoint in body_keypoints:
                    # Pad keypoint if it's not of size 3 (as done earlier)
                    if len(keypoint) < 3:
                        keypoint = keypoint + [0] * (3 - len(keypoint))
                    # Convert keypoint to tensor
                    keypoint_tensor = torch.tensor(keypoint).float()
                    # Normalize (x, y, confidence) by subtracting mean and dividing by std
                    # Add small epsilon to prevent division by zero
                    normalized_keypoint = (keypoint_tensor - mean) / (std + 1e-6)  
                    # Append the normalized keypoint to the list
                    normalized_body_keypoints.append(normalized_keypoint)
                # Append the normalized body keypoints to the person's keypoints
                normalized_person_keypoints.append(normalized_body_keypoints)
            # Append the normalized person's keypoints to the frame's keypoints
            normalized_frame_keypoints.append(normalized_person_keypoints)
        # Append the normalized frame keypoints to the main list
        normalized_keypoints.append(normalized_frame_keypoints)
    # Return the normalized keypoints
    return normalized_keypoints



In [ ]:
# Function to convert subtitle times to seconds
def time_to_float(time_str):

    # Split the time string into hours, minutes, and seconds
    hours, minutes, seconds = time_str.split(':')

    # Convert to float
    hours = float(hours)
    minutes = float(minutes)
    seconds = float(seconds)

    # Convert to total seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds

    # Return the total seconds as a float
    return total_seconds

In [ ]:
# Function to align subtitles with keypoints
def align_subtitles(normalized_keypoints, subtitles_data):
        # Set the frames per second (FPS) for the video
        fps = 25
        # Calculate the number of frames based on the length of the keypoints data
        num_frames = len(normalized_keypoints)

        # Initialize a list to hold aligned subtitles   
        aligned_subtitles = []
        # Set the initial index for subtitles
        subtitle_idx = 0
        
        # Create data structures for aligned subtitles
        aligned_subtitles = ["" for _ in range(num_frames)]
        subtitle_idx = 0

        # Iterate through each frame to align subtitles
        for frame_idx in tqdm(range(num_frames), desc="Aligning subtitles", total=num_frames, ncols=100):
            # Calculate the time for the current frame
            frame_time = frame_idx / fps

            # Check if the current subtitle is within the frame time
            while subtitle_idx < len(subtitles_data) and time_to_float(subtitles_data["end"]) < frame_time:
                # Move to the next subtitle if the current one has ended
                subtitle_idx += 1

            # Check if the current frame time is within the subtitle's start and end time
            if subtitle_idx < len(subtitles_data) and time_to_float(subtitles_data["start"]) <= frame_time <= time_to_float(subtitles_data["end"]):
                # If it is, assign the subtitle text to the aligned subtitles list
                aligned_subtitles[frame_idx] = subtitles_data["text"]

        # Only keep frames with subtitles
        relevant_frames = [idx for idx, subtitle in enumerate(aligned_subtitles) if subtitle != ""]

        print(relevant_frames)

        # Filter out normalized keypoints and subtitles for relevant frames
        normalized_keypoints = [normalized_keypoints[idx] for idx in relevant_frames]
        aligned_subtitles = [aligned_subtitles[idx] for idx in relevant_frames]

        # Return the normalized keypoints and aligned subtitles
        return normalized_keypoints, aligned_subtitles

In [ ]:
# paths to the directory containing the JSON files
test_data = load_from_json("<path_to_directory>\\test_data.json")
test_keypoints = "<path_to_directory>\\keypoints"

# Load the test data from the JSON file
filenames = []
for data in test_data:
    filenames.append(data["file"])

counter = 0

# Iterate through each folder in the directory containing the keypoints
for foldername in os.listdir("<path_to_directory>\\keypoints"):
    # Iterate through each file in the folder
    for data in test_data:
        # Check if the file name matches the folder name (excluding the last 7 characters)
        if data["file"][:-4] == foldername[:-7]:
            counter += 1
            # Build the full path to the keypoints folder
            keypoints_path = os.path.join(test_keypoints, foldername)
            print(counter , "     ", keypoints_path)
            # Unzip the tar file
            print("Unzipping tar file")
            path = extract_tar(os.path.join("D:\\bobsl\\v1.4\\video_features\\keypoints", foldername), os.path.join(test_keypoints, foldername[:-7]))
            path = os.path.join(test_keypoints, foldername[:-7])
            # Load the JSON files from the folder
            print("Combining json files")
            json_data = compile_json_from_tar(path)
            # Extract keypoints from the JSON data
            print("Extracting keypoints")
            keypoints = extract_keypoints(json_data)
            # Normalize the keypoints
            print("Normalizing keypoints")
            normalized_keypoints = normalize_keypoints(keypoints)
            # Align the subtitles with the keypoints
            print("length: ", len(keypoints))
            print("Aligning subtitles")
            aligned_keypoints, aligned_subtitles = align_subtitles(normalized_keypoints, data)
            # Save the aligned keypoints and subtitles to the specified directory
            print("Saving keypoints")
            torch.save(aligned_keypoints, os.path.join(test_keypoints, data["code"] + "_keypoints.pt"))
            print("Saving subtitles")
            torch.save(aligned_subtitles, os.path.join(test_keypoints, data["code"] + "_subtitles.pt"))


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\taylo\OneDrive\Documents\Dissertation_Work\signLanguageTranslator\preprocessing\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3579, in run_code
  File "C:\Users\taylo\AppData\Local\Temp\ipykernel_16608\2359241892.py", line 2, in <module>
    test_data = load_from_json("<path_to_directory>\\test_data.json")
                ^^^^^^^^^^^^^^
NameError: name 'load_from_json' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\taylo\OneDrive\Documents\Dissertation_Work\signLanguageTranslator\preprocessing\.venv\Lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\taylo\OneDrive\Documents\Dissertation_Work\signLanguageTranslator\prepro

In [ ]:
# paths to the directory containing the JSON files
test_data = "<path_to_directory>\\test_data.json"
test_keypoints = "<path_to_directory>\\keypoints"
test_keypoints_returned = "<path_to_directory>\\keypoints_new"

# Empty list to store filenames
filenames = []
# Load the test data from the JSON file
for data in test_data:
    filenames.append(data["file"])

counter = 0

# buffer frames to be added to the start and end of the video
buffer_frames = 0 

# frames per second (FPS) for the video
fps = 25
num_processed = 0

# Function to extract selected keypoints from the JSON data
def extract_selected_keypoints_from_folder(folder_path, start_frame, end_frame):
    # List to hold selected keypoints data
    selected_data = []
    # Iterate through the range of frames to extract
    for frame_idx in range(start_frame, end_frame):
        # Construct the filename for the keypoints JSON file
        filename = f"{foldername}_{frame_idx:012}_keypoints.json"
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        # Check if the file exists
        if os.path.exists(file_path):
            try:
                # Open and read the JSON file
                with open(file_path, 'r') as f:
                    json_frame = json.load(f)
                    selected_data.append(json_frame)
            except Exception as e:
                # Handle any errors that occur while reading the file
                print(f"[Error] Failed to read {file_path}: {e}")
        else:
            # If the file does not exist, print a warning message
            print(f"[Warning] Missing frame: {file_path}")
    # Return the selected keypoints data
    return selected_data

# Iterate through each folder in the directory containing the keypoints
for data in test_data:
    # Build the full path to the keypoints folder
    video_id = data["file"][:-4]
    foldername = video_id
    folder_path = os.path.join(test_keypoints, foldername)

    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"[Skip] Folder not found: {folder_path}")
        continue

    print(f"\n[{data['code']}] Processing video: {video_id}")
    
    # Extract the start and end times from the data
    start_sec = time_to_float(data["start"])
    end_sec = time_to_float(data["end"])
    # Calculate the start and end frames based on the FPS
    start_frame = max(int(start_sec * fps) - buffer_frames, 0)
    end_frame = int(end_sec * fps) + buffer_frames

    print(f" - Frames: {start_frame} to {end_frame}")

    # Load only the required frames
    print(" - Loading keypoint JSONs")
    json_data = extract_selected_keypoints_from_folder(folder_path, start_frame, end_frame)

    # Check if any JSON data was loaded
    if not json_data:
        print(f"[Warning] No JSON frames loaded for {data['code']} — skipping.")
        continue

    # Extract and normalize keypoints
    print(" - Extracting and normalizing keypoints")
    keypoints = extract_keypoints(json_data)
    normalized_keypoints = normalize_keypoints(keypoints)

    # Save the keypoints to the specified directory
    print(" - Saving clip keypoints and subtitle")
    save_path = os.path.join(test_keypoints_returned, data["code"] + "_keypoints.pt")
    torch.save(normalized_keypoints, save_path)

    num_processed += 1

print(f"Processed {num_processed} samples.")




[001] Processing video: 6191576750560895398
 - Frames: 65580 to 65605
 - Loading keypoint JSONs
 - Extracting and normalizing keypoints


Applying normalization: 100%|██████████████████████████████████████| 25/25 [00:00<00:00, 290.31it/s]

 - Saving clip keypoints and subtitle



[002] Processing video: 5904461193300402199
 - Frames: 80589 to 80713
 - Loading keypoint JSONs
 - Extracting and normalizing keypoints


Applying normalization: 100%|████████████████████████████████████| 124/124 [00:00<00:00, 256.60it/s]


...


[099] Processing video: 6129605526440376534
 - Frames: 64800 to 64852
 - Loading keypoint JSONs
 - Extracting and normalizing keypoints


Applying normalization: 100%|██████████████████████████████████████| 52/52 [00:00<00:00, 255.86it/s]


 - Saving clip keypoints and subtitle

[100] Processing video: 6274706272088545077
 - Frames: 63852 to 64133
 - Loading keypoint JSONs
 - Extracting and normalizing keypoints


Applying normalization: 100%|████████████████████████████████████| 281/281 [00:00<00:00, 361.33it/s]


 - Saving clip keypoints and subtitle

✅ Done. Processed 100 samples.
